# Coregistration using SPM12

Python package requirements: `pip install spm12[demo]`

In [ ]:
from __future__ import print_function
%matplotlib notebook

from os import path, getenv

from miutil.imio import nii
from miutil.plot import imscroll
import matplotlib.pyplot as plt
import numpy as np

from spm12 import regseg

HOME = getenv("DATA_ROOT", path.expanduser("~"))
DATA = path.join(HOME, "Ab_PET_mMR_test")
MRI = path.join(DATA, "T1w_N4", "t1_S00113_17598013_N4bias_cut.nii.gz")
PET = path.join(
    DATA, "testing_reference", "Ab_PET_mMR_ref", "basic", "17598013_t-3000-3600sec_itr-4_suvr.nii.gz")
if not path.exists(DATA):
    raise ValueError("""\
Cannot find Ab_PET_mMR_test in ${DATA_ROOT:-~} (%s).
Get it from https://zenodo.org/record/3877529
""" % HOME)

In [ ]:
outpath = "."
print("input PET shape:", nii.getnii(PET).shape)
print("input MRI shape:", nii.getnii(MRI).shape)
print("registering (~1min)")
reg = regseg.coreg_spm(PET, MRI, outpath=outpath)
print("affine matrix:")
print(reg["affine"])
out = regseg.resample_spm(PET, MRI, reg["affine"], outpath=outpath)

In [ ]:
def get_vol(fname):
    x = np.nan_to_num(nii.getnii(fname)[:, 100:-120, 120:-120, None])
    x -= x.min()
    x /= np.percentile(x, 99)
    x[x > 1] = 1
    return x

pet, mri = map(get_vol, [PET, out])
zer = np.zeros_like(pet)
slicer = imscroll(
    {
        "PET": np.concatenate([pet, zer, zer], axis=-1),
        "Registered MRI": np.concatenate([zer, zer, mri], axis=-1),
        "Overlay": np.concatenate([pet * 0.6, zer, mri], axis=-1),
    },
    cmaps=["Reds_r", "Blues_r", None],
    figsize=(9.5, 4),
    nrows=1, frameon=False)
slicer(70)
#plt.savefig(path.join(outpath, "pet_mr_coreg.png"))